In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
from pprint import pprint

In [3]:
config = configparser.ConfigParser()
config.read('authentication.ini')
print(config.sections())
github_oauth = config['github']['api_key']

['github']


In [6]:
project_releases = pd.read_csv('volatile_projects_complete_links_limit10_filtered.csv')

In [7]:
project_releases = project_releases[project_releases['project_name'] == 'Okhttp'].head(4)

In [8]:
project_releases.head()

,project_name,project_link,version_name,commit,timestamp
204,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.2,"Commit(sha=""b84627ef4bc43f2096c1b764720f9ff238...",30/1/2021 19:44
205,Okhttp,https://github.com/square/okhttp,parent-5.0.0-alpha.1,"Commit(sha=""79d2a8d192d007b8acb2423d3ab6bba3a4...",30/1/2021 18:36
206,Okhttp,https://github.com/square/okhttp,parent-4.10.0-RC1,"Commit(sha=""4fd1e8f99833eebdd2e99f3456322aa197...",7/10/2020 3:28
207,Okhttp,https://github.com/square/okhttp,parent-4.9.1,"Commit(sha=""63dcd95bfa2345bb3f3d4abc6b6dbf36cf...",30/1/2021 18:09


In [ ]:
project_refactoring_data = pd.read_csv('Effort_Estimation_Results/Okhttp.csv')
project_refactoring_data.head()

In [8]:
unique_releases_commit = pd.unique(project_refactoring_data['latest_commit'])
unique_commit = pd.unique(project_refactoring_data['commit_compared_with'])

In [9]:
print(unique_commit)
print(unique_releases_commit)

['3e331c108905a97fa9718b40844ddc1356fc86b5'
 'f8065acac28a0b86f3bdbd854ac87af0789416d0'
 '963c76864344d0d55ba7421f474c130a958a9649'
 '59ae0c138d6468aac5d92781b19cddc26e04e862'
 '4677beea96f0afb1e061f119e52e0203d4cd3738'
 '4036fa55b72faaad3a14f806094184ade157917d'
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8'
 '083315a473d8b2fc972a70449272b296683ac38c'
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1'
 '334eb0072512155e57a809b2fa18861c0d68ec9f']
['b84627ef4bc43f2096c1b764720f9ff2387dca32'
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1'
 '4fd1e8f99833eebdd2e99f3456322aa197f5e652']


In [10]:
list(set(unique_commit).symmetric_difference(set(unique_releases_commit)))

['4fd1e8f99833eebdd2e99f3456322aa197f5e652',
 '4036fa55b72faaad3a14f806094184ade157917d',
 '59ae0c138d6468aac5d92781b19cddc26e04e862',
 'f8065acac28a0b86f3bdbd854ac87af0789416d0',
 'b84627ef4bc43f2096c1b764720f9ff2387dca32',
 '3e331c108905a97fa9718b40844ddc1356fc86b5',
 '083315a473d8b2fc972a70449272b296683ac38c',
 '79d2a8d192d007b8acb2423d3ab6bba3a4f999e1',
 '334eb0072512155e57a809b2fa18861c0d68ec9f',
 '963c76864344d0d55ba7421f474c130a958a9649',
 '4677beea96f0afb1e061f119e52e0203d4cd3738',
 'ff959f8fd550b8f55a47038ea4235b1b2ed2eaa8',
 '5f2732de5a5bc389be2e5bd72441f2a7198b4ca1']

In [10]:
project_link = "https://github.com/square/okhttp"

In [18]:

owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/{owner}/{repo}/contributors"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/repos/square/okhttp/contributors
[{'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
  'contributions': 2592,
  'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
  'followers_url': 'https://api.github.com/users/swankjesse/followers',
  'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
  'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
  'gravatar_id': '',
  'html_url': 'https://github.com/swankjesse',
  'id': 133019,
  'login': 'swankjesse',
  'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
  'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
  'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
  'repos_url': 'https://api.github.com/users/swankjesse/repos',
  'site_admin': False,
  'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/swankjesse/subscri

In [20]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/users/swankjesse
{'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
 'bio': None,
 'blog': 'http://publicobject.com/',
 'company': 'Square, Inc.',
 'created_at': '2009-09-30T02:51:25Z',
 'email': 'jwilson@squareup.com',
 'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
 'followers': 5667,
 'followers_url': 'https://api.github.com/users/swankjesse/followers',
 'following': 0,
 'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
 'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': None,
 'html_url': 'https://github.com/swankjesse',
 'id': 133019,
 'location': 'Waterloo, Ontario, Canada',
 'login': 'swankjesse',
 'name': 'Jesse Wilson',
 'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
 'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
 'public_gists': 91,
 'public_repos': 22,
 'received_events_url': 'https://api.github.com/users/swankj

In [24]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
query_url = "https://api.github.com/users/swankjesse/repos"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

print(len(r.json()))

https://api.github.com/users/swankjesse/repos
[{'allow_forking': True,
  'archive_url': 'https://api.github.com/repos/swankjesse/compose-samples/{archive_format}{/ref}',
  'archived': False,
  'assignees_url': 'https://api.github.com/repos/swankjesse/compose-samples/assignees{/user}',
  'blobs_url': 'https://api.github.com/repos/swankjesse/compose-samples/git/blobs{/sha}',
  'branches_url': 'https://api.github.com/repos/swankjesse/compose-samples/branches{/branch}',
  'clone_url': 'https://github.com/swankjesse/compose-samples.git',
  'collaborators_url': 'https://api.github.com/repos/swankjesse/compose-samples/collaborators{/collaborator}',
  'comments_url': 'https://api.github.com/repos/swankjesse/compose-samples/comments{/number}',
  'commits_url': 'https://api.github.com/repos/swankjesse/compose-samples/commits{/sha}',
  'compare_url': 'https://api.github.com/repos/swankjesse/compose-samples/compare/{base}...{head}',
  'contents_url': 'https://api.github.com/repos/swankjesse/compos

  'open_issues': 0,
  'open_issues_count': 0,
  'owner': {'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
            'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
            'followers_url': 'https://api.github.com/users/swankjesse/followers',
            'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
            'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
            'gravatar_id': '',
            'html_url': 'https://github.com/swankjesse',
            'id': 133019,
            'login': 'swankjesse',
            'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
            'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
            'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
            'repos_url': 'https://api.github.com/users/swankjesse/repos',
            'site_admin': False,
            'starred_url': 'https://api.github.co

            'site_admin': False,
            'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
            'subscriptions_url': 'https://api.github.com/users/swankjesse/subscriptions',
            'type': 'User',
            'url': 'https://api.github.com/users/swankjesse'},
  'permissions': {'admin': False,
                  'maintain': False,
                  'pull': True,
                  'push': False,
                  'triage': False},
  'private': False,
  'pulls_url': 'https://api.github.com/repos/swankjesse/micronaut-core/pulls{/number}',
  'pushed_at': '2021-02-09T04:27:36Z',
  'releases_url': 'https://api.github.com/repos/swankjesse/micronaut-core/releases{/id}',
  'size': 69769,
  'ssh_url': 'git@github.com:swankjesse/micronaut-core.git',
  'stargazers_count': 0,
  'stargazers_url': 'https://api.github.com/repos/swankjesse/micronaut-core/stargazers',
  'statuses_url': 'https://api.github.com/repos/swankjesse/micronaut-core/statuses/{sha}',
 

In [23]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/users/swankjesse"
query_url = 'https://api.github.com/repos/swankjesse/protobuf/languages'
query_url = 'https://api.github.com/repos/swankjesse/compose-samples/languages'
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "open",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())

https://api.github.com/repos/swankjesse/compose-samples/languages
{'Kotlin': 157439, 'Shell': 1102}


In [ ]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

In [30]:

r = requests.get("https://api.github.com/search/code?q=addClass+user:mozilla",headers=headers)
pprint(r.json())

{'incomplete_results': False,
 'items': [{'git_url': 'https://api.github.com/repositories/10508666/git/blobs/32e18fbecdf1eee3dcda3e7877ea60bdaae4de06',
            'html_url': 'https://github.com/mozilla/picl-mocks/blob/7478d06de1977a1ad406135015d0bb7ee4f7b692/public/javascripts/one.js',
            'name': 'one.js',
            'path': 'public/javascripts/one.js',
            'repository': {'archive_url': 'https://api.github.com/repos/mozilla/picl-mocks/{archive_format}{/ref}',
                           'assignees_url': 'https://api.github.com/repos/mozilla/picl-mocks/assignees{/user}',
                           'blobs_url': 'https://api.github.com/repos/mozilla/picl-mocks/git/blobs{/sha}',
                           'branches_url': 'https://api.github.com/repos/mozilla/picl-mocks/branches{/branch}',
                           'collaborators_url': 'https://api.github.com/repos/mozilla/picl-mocks/collaborators{/collaborator}',
                           'comments_url': 'https://api.g

                                     'subscriptions_url': 'https://api.github.com/users/mozilla/subscriptions',
                                     'type': 'Organization',
                                     'url': 'https://api.github.com/users/mozilla'},
                           'private': False,
                           'pulls_url': 'https://api.github.com/repos/mozilla/remo/pulls{/number}',
                           'releases_url': 'https://api.github.com/repos/mozilla/remo/releases{/id}',
                           'stargazers_url': 'https://api.github.com/repos/mozilla/remo/stargazers',
                           'statuses_url': 'https://api.github.com/repos/mozilla/remo/statuses/{sha}',
                           'subscribers_url': 'https://api.github.com/repos/mozilla/remo/subscribers',
                           'subscription_url': 'https://api.github.com/repos/mozilla/remo/subscription',
                           'tags_url': 'https://api.github.com/repos/mozilla/remo/t

                           'url': 'https://api.github.com/repos/mozilla/popcorn-editor'},
            'score': 1.0,
            'sha': '7130282b08532c112e461d6abda8b18064544c98',
            'url': 'https://api.github.com/repositories/36961669/contents/PopcornEditor/static/bower/bootstrap/js/collapse.js?ref=901c5c65faee66b024a89f5f6324ae20c653bafb'},
           {'git_url': 'https://api.github.com/repositories/8780590/git/blobs/18d8009f0f2a44dbac08f2b6f7af23fe6b5624d5',
            'html_url': 'https://github.com/mozilla/webmaker.org/blob/dc87802c9d4122550768c2adfad0c31fcab2f717/public/js/base/login.js',
            'name': 'login.js',
            'path': 'public/js/base/login.js',
            'repository': {'archive_url': 'https://api.github.com/repos/mozilla/webmaker.org/{archive_format}{/ref}',
                           'assignees_url': 'https://api.github.com/repos/mozilla/webmaker.org/assignees{/user}',
                           'blobs_url': 'https://api.github.com/repos/mozilla/

In [19]:
r.links

{'next': {'url': 'https://api.github.com/repositories/5152285/commits?page=31&per_page=100',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/5152285/commits?page=50&per_page=100',
  'rel': 'last'},
 'first': {'url': 'https://api.github.com/repositories/5152285/commits?page=1&per_page=100',
  'rel': 'first'},
 'prev': {'url': 'https://api.github.com/repositories/5152285/commits?page=29&per_page=100',
  'rel': 'prev'}}

In [18]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/commits?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers)
pprint(r.json())
print(len(r.json()))

https://api.github.com/repos/square/okhttp/commits?page=30&per_page=100
[{'author': {'avatar_url': 'https://avatars.githubusercontent.com/u/133019?v=4',
             'events_url': 'https://api.github.com/users/swankjesse/events{/privacy}',
             'followers_url': 'https://api.github.com/users/swankjesse/followers',
             'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
             'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
             'gravatar_id': '',
             'html_url': 'https://github.com/swankjesse',
             'id': 133019,
             'login': 'swankjesse',
             'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
             'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
             'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
             'repos_url': 'https://api.github.com/users/swankjesse/repos',
             'site_admin': False,
         

             'type': 'User',
             'url': 'https://api.github.com/users/iagreen'},
  'comments_url': 'https://api.github.com/repos/square/okhttp/commits/597c279dfe51daf30457da85cc033feb34325286/comments',
  'commit': {'author': {'date': '2015-09-09T12:47:09Z',
                        'email': 'isaac@wink.com',
                        'name': 'Isaac Green'},
             'comment_count': 0,
             'committer': {'date': '2015-09-09T13:07:00Z',
                           'email': 'isaac@wink.com',
                           'name': 'Isaac Green'},
             'message': 'only encode URI problem chars when actually '
                        'converting to URI',
             'tree': {'sha': '4f158072c7d888de598722987356d8f664431cf9',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/4f158072c7d888de598722987356d8f664431cf9'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/597c279dfe51daf30457da85cc033feb34325286',
 

             'repos_url': 'https://api.github.com/users/JakeWharton/repos',
             'site_admin': False,
             'starred_url': 'https://api.github.com/users/JakeWharton/starred{/owner}{/repo}',
             'subscriptions_url': 'https://api.github.com/users/JakeWharton/subscriptions',
             'type': 'User',
             'url': 'https://api.github.com/users/JakeWharton'},
  'comments_url': 'https://api.github.com/repos/square/okhttp/commits/22c6eb7ecb4a3e6115f70f5f3c1c5c4e0d49b9ab/comments',
  'commit': {'author': {'date': '2015-08-26T01:21:40Z',
                        'email': 'jakewharton@gmail.com',
                        'name': 'Jake Wharton'},
             'comment_count': 0,
             'committer': {'date': '2015-08-26T01:21:40Z',
                           'email': 'jakewharton@gmail.com',
                           'name': 'Jake Wharton'},
             'message': '2.5.0 is latest.',
             'tree': {'sha': '4ac8683e7fd701a3fb9fd9cac1abcf6d2188faba',
  

  'commit': {'author': {'date': '2015-08-03T05:32:13Z',
                        'email': 'jesse@swank.ca',
                        'name': 'Jesse Wilson'},
             'comment_count': 0,
             'committer': {'date': '2015-08-03T05:32:13Z',
                           'email': 'jesse@swank.ca',
                           'name': 'Jesse Wilson'},
             'message': 'Merge pull request #1786 from '
                        'square/jwilson_0803_default_user_agent\n'
                        '\n'
                        'Make sure the default user agent is ASCII.',
             'tree': {'sha': 'c2f50df63779ec60daa01682e176274ed7b51502',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/c2f50df63779ec60daa01682e176274ed7b51502'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/e19b7913eb0ec73a24872c0abce85896728a9648',
             'verification': {'payload': None,
                              'reason': 'unsigned',
     

                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/5576502b317b28618ce79f41f958364f2e737661'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/8f5e344aba7b734efa61977651838a88d288a187',
             'verification': {'payload': None,
                              'reason': 'unsigned',
                              'signature': None,
                              'verified': False}},
  'committer': {'avatar_url': 'https://avatars.githubusercontent.com/u/66577?v=4',
                'events_url': 'https://api.github.com/users/JakeWharton/events{/privacy}',
                'followers_url': 'https://api.github.com/users/JakeWharton/followers',
                'following_url': 'https://api.github.com/users/JakeWharton/following{/other_user}',
                'gists_url': 'https://api.github.com/users/JakeWharton/gists{/gist_id}',
                'gravatar_id': '',
                'html_url': 'https://github.com/JakeWharton',
        

             'following_url': 'https://api.github.com/users/swankjesse/following{/other_user}',
             'gists_url': 'https://api.github.com/users/swankjesse/gists{/gist_id}',
             'gravatar_id': '',
             'html_url': 'https://github.com/swankjesse',
             'id': 133019,
             'login': 'swankjesse',
             'node_id': 'MDQ6VXNlcjEzMzAxOQ==',
             'organizations_url': 'https://api.github.com/users/swankjesse/orgs',
             'received_events_url': 'https://api.github.com/users/swankjesse/received_events',
             'repos_url': 'https://api.github.com/users/swankjesse/repos',
             'site_admin': False,
             'starred_url': 'https://api.github.com/users/swankjesse/starred{/owner}{/repo}',
             'subscriptions_url': 'https://api.github.com/users/swankjesse/subscriptions',
             'type': 'User',
             'url': 'https://api.github.com/users/swankjesse'},
  'comments_url': 'https://api.github.com/repos/square

                           'name': 'jwilson'},
             'message': 'Modify timeout exception / update test expectations\n'
                        '\n'
                        'Related to issue #1676.\n'
                        'Depends on https://github.com/square/okio/pull/154\n'
                        '\n'
                        'URLConnection should throw SocketTimeoutException for '
                        'timeouts\n'
                        'not InterruptedIOException.',
             'tree': {'sha': '33d682684b74733aab643618000e04381bb3ac09',
                      'url': 'https://api.github.com/repos/square/okhttp/git/trees/33d682684b74733aab643618000e04381bb3ac09'},
             'url': 'https://api.github.com/repos/square/okhttp/git/commits/5af1d7f449509597f16bebbf0ef6c68d7c924398',
             'verification': {'payload': None,
                              'reason': 'unsigned',
                              'signature': None,
                              'verified': Fa

In [3]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

https://api.github.com/repos/square/okhttp/issues?page=30&per_page=100
[{'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'COLLABORATOR',
  'body': 'This might have been a difference between SPDY and HTTP/2. '
          'Reviewing the\r\n'
          'HTTP/2 spec, only a WINDOWS_UPDATE frame can change the '
          'flow-control\r\n'
          'window for a connection.',
  'closed_at': '2018-03-20T02:36:24Z',
  'comments': 3,
  'comments_url': 'https://api.github.com/repos/square/okhttp/issues/3920/comments',
  'created_at': '2018-03-12T14:52:29Z',
  'draft': False,
  'events_url': 'https://api.github.com/repos/square/okhttp/issues/3920/events',
  'html_url': 'https://github.com/square/okhttp/pull/3920',
  'id': 304404099,
  'labels': [],
  'labels_url': 'https://api.github.com/repos/square/okhttp/issues/3920/labels{/name}',
  'locked': False,
  'milestone': None,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0MTc0MzkyMjIy',
  'number': 3920,
  'performed

          'The current connection pool constructor only allows max idle '
          "connections and timeout for the life of the connection. Didn't let "
          'me set a pool size. At least tell me what the default pool size is '
          'or how big it can get\r\n'
          '\r\n'
          'https://stackoverflow.com/questions/49069297/okhttpclient-connection-pool-size-dilemma/49070993#49070993',
  'closed_at': '2018-03-03T12:16:16Z',
  'comments': 2,
  'comments_url': 'https://api.github.com/repos/square/okhttp/issues/3907/comments',
  'created_at': '2018-03-03T11:19:12Z',
  'events_url': 'https://api.github.com/repos/square/okhttp/issues/3907/events',
  'html_url': 'https://github.com/square/okhttp/issues/3907',
  'id': 301990338,
  'labels': [],
  'labels_url': 'https://api.github.com/repos/square/okhttp/issues/3907/labels{/name}',
  'locked': False,
  'milestone': None,
  'node_id': 'MDU6SXNzdWUzMDE5OTAzMzg=',
  'number': 3907,
  'performed_via_github_app': None,
  'reaction

                'url': 'https://api.github.com/repos/square/okhttp/issues/3893/reactions'},
  'repository_url': 'https://api.github.com/repos/square/okhttp',
  'state': 'closed',
  'timeline_url': 'https://api.github.com/repos/square/okhttp/issues/3893/timeline',
  'title': 'IllegalStateException thrown from AsyncTimeout in Android 4.4.2',
  'updated_at': '2018-02-26T22:00:14Z',
  'url': 'https://api.github.com/repos/square/okhttp/issues/3893',
  'user': {'avatar_url': 'https://avatars.githubusercontent.com/u/3209546?v=4',
           'events_url': 'https://api.github.com/users/jorgeper/events{/privacy}',
           'followers_url': 'https://api.github.com/users/jorgeper/followers',
           'following_url': 'https://api.github.com/users/jorgeper/following{/other_user}',
           'gists_url': 'https://api.github.com/users/jorgeper/gists{/gist_id}',
           'gravatar_id': '',
           'html_url': 'https://github.com/jorgeper',
           'id': 3209546,
           'login': 'jorge

  'node_id': 'MDExOlB1bGxSZXF1ZXN0MTcwNjMwNzQ0',
  'number': 3877,
  'performed_via_github_app': None,
  'pull_request': {'diff_url': 'https://github.com/square/okhttp/pull/3877.diff',
                   'html_url': 'https://github.com/square/okhttp/pull/3877',
                   'merged_at': '2018-02-22T01:16:14Z',
                   'patch_url': 'https://github.com/square/okhttp/pull/3877.patch',
                   'url': 'https://api.github.com/repos/square/okhttp/pulls/3877'},
  'reactions': {'+1': 0,
                '-1': 0,
                'confused': 0,
                'eyes': 0,
                'heart': 0,
                'hooray': 0,
                'laugh': 0,
                'rocket': 0,
                'total_count': 0,
                'url': 'https://api.github.com/repos/square/okhttp/issues/3877/reactions'},
  'repository_url': 'https://api.github.com/repos/square/okhttp',
  'state': 'closed',
  'timeline_url': 'https://api.github.com/repos/square/okhttp/issues/3877/timel

                'description': '',
                'due_on': '2019-03-14T07:00:00Z',
                'html_url': 'https://github.com/square/okhttp/milestone/30',
                'id': 4024076,
                'labels_url': 'https://api.github.com/repos/square/okhttp/milestones/30/labels',
                'node_id': 'MDk6TWlsZXN0b25lNDAyNDA3Ng==',
                'number': 30,
                'open_issues': 0,
                'state': 'closed',
                'title': '3.14',
                'updated_at': '2019-03-14T11:30:31Z',
                'url': 'https://api.github.com/repos/square/okhttp/milestones/30'},
  'node_id': 'MDU6SXNzdWUyOTgzODc4MTA=',
  'number': 3865,
  'performed_via_github_app': None,
  'reactions': {'+1': 0,
                '-1': 0,
                'confused': 0,
                'eyes': 0,
                'heart': 0,
                'hooray': 0,
                'laugh': 0,
                'rocket': 0,
                'total_count': 0,
                'url': 'https:

                'eyes': 0,
                'heart': 0,
                'hooray': 0,
                'laugh': 0,
                'rocket': 0,
                'total_count': 0,
                'url': 'https://api.github.com/repos/square/okhttp/issues/3860/reactions'},
  'repository_url': 'https://api.github.com/repos/square/okhttp',
  'state': 'closed',
  'timeline_url': 'https://api.github.com/repos/square/okhttp/issues/3860/timeline',
  'title': 'Proguard whining about javax.annotation.Nonnull',
  'updated_at': '2018-02-18T16:42:30Z',
  'url': 'https://api.github.com/repos/square/okhttp/issues/3860',
  'user': {'avatar_url': 'https://avatars.githubusercontent.com/u/119509?v=4',
           'events_url': 'https://api.github.com/users/sevar83/events{/privacy}',
           'followers_url': 'https://api.github.com/users/sevar83/followers',
           'following_url': 'https://api.github.com/users/sevar83/following{/other_user}',
           'gists_url': 'https://api.github.com/users/sevar83/

  'timeline_url': 'https://api.github.com/repos/square/okhttp/issues/3846/timeline',
  'title': 'Transfer to/from FileChannel directly from Buffer.',
  'updated_at': '2018-03-14T04:35:20Z',
  'url': 'https://api.github.com/repos/square/okhttp/issues/3846',
  'user': {'avatar_url': 'https://avatars.githubusercontent.com/u/66577?v=4',
           'events_url': 'https://api.github.com/users/JakeWharton/events{/privacy}',
           'followers_url': 'https://api.github.com/users/JakeWharton/followers',
           'following_url': 'https://api.github.com/users/JakeWharton/following{/other_user}',
           'gists_url': 'https://api.github.com/users/JakeWharton/gists{/gist_id}',
           'gravatar_id': '',
           'html_url': 'https://github.com/JakeWharton',
           'id': 66577,
           'login': 'JakeWharton',
           'node_id': 'MDQ6VXNlcjY2NTc3',
           'organizations_url': 'https://api.github.com/users/JakeWharton/orgs',
           'received_events_url': 'https://api.gi

                'url': 'https://api.github.com/repos/square/okhttp/issues/3831/reactions'},
  'repository_url': 'https://api.github.com/repos/square/okhttp',
  'state': 'closed',
  'timeline_url': 'https://api.github.com/repos/square/okhttp/issues/3831/timeline',
  'title': '[Feature Request] allow supplying a different cache key derivation '
           'function for urls',
  'updated_at': '2020-03-29T18:00:19Z',
  'url': 'https://api.github.com/repos/square/okhttp/issues/3831',
  'user': {'avatar_url': 'https://avatars.githubusercontent.com/u/35414275?v=4',
           'events_url': 'https://api.github.com/users/2fchar/events{/privacy}',
           'followers_url': 'https://api.github.com/users/2fchar/followers',
           'following_url': 'https://api.github.com/users/2fchar/following{/other_user}',
           'gists_url': 'https://api.github.com/users/2fchar/gists{/gist_id}',
           'gravatar_id': '',
           'html_url': 'https://github.com/2fchar',
           'id': 35414275,

           'site_admin': False,
           'starred_url': 'https://api.github.com/users/yschimke/starred{/owner}{/repo}',
           'subscriptions_url': 'https://api.github.com/users/yschimke/subscriptions',
           'type': 'User',
           'url': 'https://api.github.com/users/yschimke'}},
 {'active_lock_reason': None,
  'assignee': None,
  'assignees': [],
  'author_association': 'COLLABORATOR',
  'body': '',
  'closed_at': '2018-02-04T20:52:04Z',
  'comments': 6,
  'comments_url': 'https://api.github.com/repos/square/okhttp/issues/3821/comments',
  'created_at': '2018-02-04T19:55:23Z',
  'draft': False,
  'events_url': 'https://api.github.com/repos/square/okhttp/issues/3821/events',
  'html_url': 'https://github.com/square/okhttp/pull/3821',
  'id': 294225029,
  'labels': [],
  'labels_url': 'https://api.github.com/repos/square/okhttp/issues/3821/labels{/name}',
  'locked': False,
  'milestone': None,
  'node_id': 'MDExOlB1bGxSZXF1ZXN0MTY3MDI2NjQy',
  'number': 3821,
  'perform

In [17]:
owner = "MartinHeinz"
repo = "python-project-blueprint"

owner = 'apache'
repo = "camel"
commit_sha = 'fde51bfdce52572ee000e383293f2be42d62f22f'
query_url = f"https://api.github.com/repos/{owner}/{repo}/commits/{commit_sha}"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

https://api.github.com/repos/apache/camel/commits/fde51bfdce52572ee000e383293f2be42d62f22f
{'author': {'avatar_url': 'https://avatars.githubusercontent.com/u/8310026?v=4',
            'events_url': 'https://api.github.com/users/Paul-Blanchaert/events{/privacy}',
            'followers_url': 'https://api.github.com/users/Paul-Blanchaert/followers',
            'following_url': 'https://api.github.com/users/Paul-Blanchaert/following{/other_user}',
            'gists_url': 'https://api.github.com/users/Paul-Blanchaert/gists{/gist_id}',
            'gravatar_id': '',
            'html_url': 'https://github.com/Paul-Blanchaert',
            'id': 8310026,
            'login': 'Paul-Blanchaert',
            'node_id': 'MDQ6VXNlcjgzMTAwMjY=',
            'organizations_url': 'https://api.github.com/users/Paul-Blanchaert/orgs',
            'received_events_url': 'https://api.github.com/users/Paul-Blanchaert/received_events',
            'repos_url': 'https://api.github.com/users/Paul-Blanchaer

                     'Optional.ofNullable(zkChroot);\n'
                     '+        CloudSolrClient.Builder builder = new '
                     'CloudSolrClient.Builder(urlList, zkChrootOptional);\n'
                     '+        if (connectionTimeout != null) {\n'
                     '+            '
                     'builder.withConnectionTimeout(connectionTimeout);\n'
                     '+        }\n'
                     '+        if (soTimeout != null) {\n'
                     '+            builder.withSocketTimeout(soTimeout);\n'
                     '+        }\n'
                     '+        if (httpClient != null) {\n'
                     '+            builder.withHttpClient(httpClient);\n'
                     '+        }\n'
                     '+        CloudSolrClient cloudSolrClient = '
                     'builder.build();\n'
                     '+        if (collection != null && '
                     '!collection.isEmpty()) {\n'
                     '

                     'allowCompression);\n'
                     '             return this;\n'
                     '         }\n'
                     '         /**\n'
                     '-         * Set the ZooKeeper host information which the '
                     'solrCloud could use,\n'
                     '-         * such as zkhost=localhost:8123.\n'
                     '+         * Indicates whether redirects are used to get '
                     'to the Solr server.\n'
                     '+         * \n'
                     '+         * The option is a: '
                     '&lt;code&gt;java.lang.Boolean&lt;/code&gt; type.\n'
                     '+         * \n'
                     '+         * Group: HttpSolrClient\n'
                     '+         * \n'
                     '+         * @param followRedirects the value to set\n'
                     '+         * @return the dsl builder\n'
                     '+         */\n'
                     '+        defa

In [5]:
r.json()['stats']

{'total': 39, 'additions': 34, 'deletions': 5}

In [19]:
import json
with open('raw_issues_github_json/Okhttp.json', 'w', encoding='utf-8') as f:
    json.dump(r.json(), f, ensure_ascii=False, indent=4)

In [22]:
with open('raw_issues_github_json/Okhttp.json',) as f:
    data = json.load(f)

print(data)
print(data['resources'])

{'resources': {'core': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1636302250}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'source_import': {'limit': 100, 'used': 0, 'remaining': 100, 'reset': 1636302250}, 'code_scanning_upload': {'limit': 500, 'used': 0, 'remaining': 500, 'reset': 1636305790}, 'actions_runner_registration': {'limit': 10000, 'used': 0, 'remaining': 10000, 'reset': 1636305790}}, 'rate': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}}
{'core': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'search': {'limit': 30, 'used': 0, 'remaining': 30, 'reset': 1636302250}, 'graphql': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636305790}, 'integration_manifest': {'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1636

In [20]:
owner = 'square'
repo = "okhttp"
query_url = f"https://api.github.com/rate_limit"
print(query_url)
#query_url = f"https://api.github.com/repos/{repo}"
params = {
    "state": "closed",
}
headers = {'Authorization': f'token {github_oauth}'}
r = requests.get(query_url, headers=headers, params=params)
pprint(r.json())
print(len(r.json()))

https://api.github.com/rate_limit
{'rate': {'limit': 5000, 'remaining': 4956, 'reset': 1636356890, 'used': 44},
 'resources': {'actions_runner_registration': {'limit': 10000,
                                               'remaining': 10000,
                                               'reset': 1636360178,
                                               'used': 0},
               'code_scanning_upload': {'limit': 500,
                                        'remaining': 500,
                                        'reset': 1636360178,
                                        'used': 0},
               'core': {'limit': 5000,
                        'remaining': 4956,
                        'reset': 1636356890,
                        'used': 44},
               'graphql': {'limit': 5000,
                           'remaining': 5000,
                           'reset': 1636360178,
                           'used': 0},
               'integration_manifest': {'limit': 5000,
            